# Transformaciones precios
En este notebook se harán las transformaciones que se discuten en el notebook de exploración de datasets sobre los dataset de precios.

In [3]:
import pandas as pd
import unidecode
import numpy as np


### precios_semana_20200413.csv

In [3]:
precios_0413 = pd.read_csv('../datasets/precios_semana_20200413.csv')
precios_0413.head()

,precio,producto_id,sucursal_id
0,29.90,0000000001663,2-1-014
1,29.90,0000000002288,2-1-032
2,39.90,0000000002288,2-1-096
3,499.99,0000000205870,9-1-686
4,519.99,0000000205870,9-2-248


In [4]:
# hacemos tabla auxiliar para los nulos
precios_aux = precios_0413.loc[(precios_0413.producto_id.isnull()) | (precios_0413.sucursal_id.isnull())].copy()
precios_aux['tipoError'] = 0 
precios_aux

,precio,producto_id,sucursal_id,tipoError
246393,NaN,NaN,NaN,0
246748,NaN,NaN,NaN,0
246882,NaN,NaN,NaN,0
246978,NaN,NaN,NaN,0
247033,NaN,NaN,NaN,0
247079,451.00,NaN,NaN,0
247167,NaN,NaN,NaN,0
247310,NaN,NaN,NaN,0
247404,NaN,NaN,NaN,0
247405,NaN,NaN,NaN,0


In [5]:
# Dropeamos nulos 
index_nulls = precios_0413.loc[(precios_0413.producto_id.isnull()) | (precios_0413.sucursal_id.isnull())].index.to_list()
precios_0413.drop(index_nulls, axis='index', inplace=True)
# revisamos que no hayan nulos
precios_0413.loc[(precios_0413.producto_id.isnull()) | (precios_0413.sucursal_id.isnull())]

,precio,producto_id,sucursal_id


In [6]:
# hay productos con los productos id de más de 13 digitos
precios_0413.loc[precios_0413.producto_id.str.len() != 13]

,precio,producto_id,sucursal_id
11540,59.00,10-1-2300073000005,10-1-25
11541,7.49,10-1-2300073000005,10-1-48
11542,59.00,10-1-2300075000003,10-1-25
11543,279.90,10-1-2300160000000,10-1-8
11544,1274.00,10-1-2300175000002,10-1-6
...,...,...,...
472008,387.00,9-3-0000000994002,9-3-5222
472009,509.00,9-3-0000000995696,9-3-628
472010,90.99,9-3-0000000997096,9-3-5222
472011,219.00,9-3-0000000997126,9-3-5222


In [7]:
# por ahora parece estar bien
precios_0413.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 472151 entries, 0 to 472165
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   precio       472151 non-null  float64
 1   producto_id  472151 non-null  object 
 2   sucursal_id  472151 non-null  object 
dtypes: float64(1), object(2)
memory usage: 14.4+ MB


In [8]:
# le agregamos el idSemana
filename = 'precios_semana_20200413.csv'
# Lo obtenemos del nombre del archivo
filename_split = filename.split('_')[2].split('.')
semana = filename_split[0]
tipo = filename_split[1]

In [9]:
precios_0413['semanaId'] = semana
precios_0413

,precio,producto_id,sucursal_id,semanaId
0,29.90,0000000001663,2-1-014,20200413
1,29.90,0000000002288,2-1-032,20200413
2,39.90,0000000002288,2-1-096,20200413
3,499.99,0000000205870,9-1-686,20200413
4,519.99,0000000205870,9-2-248,20200413
...,...,...,...,...
472161,139.99,9569753142128,25-1-1,20200413
472162,34.99,9795403001143,25-1-1,20200413
472163,312.50,9990385651922,5-1-3,20200413
472164,312.50,9990385651939,5-1-3,20200413


In [10]:
# Agrego id y reordeno
precios_0413.reset_index(drop=True, inplace=True)
precios_0413['precioId'] = precios_0413.index + 1
precios_0413 = precios_0413[['precioId','precio','producto_id','sucursal_id','semanaId']].copy()
precios_0413.rename({'producto_id':'productoId','sucursal_id':'sucursalId'}, axis='columns', inplace=True)
precios_0413

,precioId,precio,productoId,sucursalId,semanaId
0,1,29.90,0000000001663,2-1-014,20200413
1,2,29.90,0000000002288,2-1-032,20200413
2,3,39.90,0000000002288,2-1-096,20200413
3,4,499.99,0000000205870,9-1-686,20200413
4,5,519.99,0000000205870,9-2-248,20200413
...,...,...,...,...,...
472146,472147,139.99,9569753142128,25-1-1,20200413
472147,472148,34.99,9795403001143,25-1-1,20200413
472148,472149,312.50,9990385651922,5-1-3,20200413
472149,472150,312.50,9990385651939,5-1-3,20200413


In [11]:
# cambio los indices de producto por los nuevos
prod_aux = pd.read_csv('../datasets/producto_auxiliar.csv') #se genera en transformaciones_producto.ipynb
precios_0413 = pd.merge(precios_0413, prod_aux, left_on = 'productoId', right_on = 'antiguoId', how = 'left')
precios_0413.drop(['productoId_x','antiguoId'], axis=1, inplace=True)
precios_0413.rename({'productoId_y':'productoId'}, axis='columns', inplace=True)
precios_0413 = precios_0413[['precioId','precio','productoId','sucursalId','semanaId']].copy()
precios_0413

,precioId,precio,productoId,sucursalId,semanaId
0,1,29.90,1,2-1-014,20200413
1,2,29.90,2,2-1-032,20200413
2,3,39.90,2,2-1-096,20200413
3,4,499.99,462,9-1-686,20200413
4,5,519.99,462,9-2-248,20200413
...,...,...,...,...,...
472146,472147,139.99,26084,25-1-1,20200413
472147,472148,34.99,48301,25-1-1,20200413
472148,472149,312.50,68184,5-1-3,20200413
472149,472150,312.50,68185,5-1-3,20200413


In [12]:
# Guardo a un csv
precios_0413.to_csv('../datasets/precios_0413.csv', index=False)

### precios_semanas_20200419_20200426.xlsx

In [4]:
xls = pd.ExcelFile('../datasets/precios_semanas_20200419_20200426.xlsx')
precios_0426 = pd.read_excel(xls, 'precios_20200426_20200426')
precios_0419 = pd.read_excel(xls, 'precios_20200419_20200419')

In [6]:
precios_0419.to_csv('../datasets/precios_semanas_20200419.csv', index=False)
precios_0426.to_csv('../datasets/precios_semanas_20200426.csv', index=False)

In [23]:
filename = 'precios_semanas_20200419_20200426.xlsx'

semana_0419 = filename.split('_')[2]
filename_split = filename.split('_')[3].split('.')
semana_0426 = filename_split[0]
tipo = filename_split[1]

In [24]:
precios_0419

,precio,sucursal_id,producto_id
0,29.90,2-1-184,2288.0
1,39.90,2-1-206,2288.0
2,499.99,9-1-430,205870.0
3,539.99,9-2-107,205870.0
4,539.99,5218-03-09 00:00:00,205870.0
...,...,...,...
458538,139.99,2001-01-25 00:00:00,9569753142128.0
458539,34.99,2001-01-25 00:00:00,9795403001143.0
458540,312.50,2003-01-05 00:00:00,9990385651922.0
458541,312.50,2003-01-05 00:00:00,9990385651939.0


In [25]:
type(precios_0419.producto_id[0])

float

In [26]:
# Arreglamos el producto_id para que sea un codigo EAN
precios_0419.producto_id = precios_0419.producto_id.astype(str).str.replace('.0', '', regex=False)
precios_0419.producto_id = precios_0419.producto_id.str.zfill(13)
precios_0419.head()

,precio,sucursal_id,producto_id
0,29.90,2-1-184,0000000002288
1,39.90,2-1-206,0000000002288
2,499.99,9-1-430,0000000205870
3,539.99,9-2-107,0000000205870
4,539.99,5218-03-09 00:00:00,0000000205870


In [27]:
'''
# Hay timestamps en sucursal_id
precios_0419.sucursal_id = precios_0419.sucursal_id.astype(str)
#precios_0419.sucursal_id.apply(lambda col: col.find('00:00:00'))
# Hay muchas sucursales con timestamp 
precios_0419.loc[precios_0419.sucursal_id.str.split(expand=True)[1].notnull(),'sucursal_id'] = 'Sin dato'
'''

"\n# Hay timestamps en sucursal_id\nprecios_0419.sucursal_id = precios_0419.sucursal_id.astype(str)\n#precios_0419.sucursal_id.apply(lambda col: col.find('00:00:00'))\n# Hay muchas sucursales con timestamp \nprecios_0419.loc[precios_0419.sucursal_id.str.split(expand=True)[1].notnull(),'sucursal_id'] = 'Sin dato'\n"

In [28]:
# le agregamos el codigo de semana 
precios_0419['semanaId'] = semana_0419
precios_0419

,precio,sucursal_id,producto_id,semanaId
0,29.90,2-1-184,0000000002288,20200419
1,39.90,2-1-206,0000000002288,20200419
2,499.99,9-1-430,0000000205870,20200419
3,539.99,9-2-107,0000000205870,20200419
4,539.99,5218-03-09 00:00:00,0000000205870,20200419
...,...,...,...,...
458538,139.99,2001-01-25 00:00:00,9569753142128,20200419
458539,34.99,2001-01-25 00:00:00,9795403001143,20200419
458540,312.50,2003-01-05 00:00:00,9990385651922,20200419
458541,312.50,2003-01-05 00:00:00,9990385651939,20200419


In [29]:
precios_0419.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 458543 entries, 0 to 458542
Data columns (total 4 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   precio       456736 non-null  float64
 1   sucursal_id  458543 non-null  object 
 2   producto_id  458543 non-null  object 
 3   semanaId     458543 non-null  object 
dtypes: float64(1), object(3)
memory usage: 14.0+ MB


### precios_semana_20200503.json

In [7]:
precios_0503 = pd.read_json('../datasets/precios_semana_20200503.json')
precios_0503.head()

,precio,producto_id,sucursal_id
0,29.9,0000000002288,2-1-187
1,39.9,0000000002288,2-3-247
2,499.99,0000000205870,9-1-685
3,539.99,0000000205870,9-2-22
4,519.99,0000000205870,9-2-59


In [31]:
precios_0503.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 397734 entries, 0 to 397733
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   precio       397734 non-null  object
 1   producto_id  397734 non-null  object
 2   sucursal_id  397734 non-null  object
dtypes: object(3)
memory usage: 9.1+ MB


In [32]:
precios_0503.producto_id = precios_0503.producto_id.astype(str)

In [33]:
# revisamos los idproducto
precios_0503.loc[precios_0503.producto_id.str.len() != 13]

,precio,producto_id,sucursal_id
9649,399.9,1-1-1500210000009,1-1-7
9650,522,1-1-2270004000001,1-1-7
9651,522,1-1-2270005000000,1-1-7
9652,690.9,1-1-2270006000009,1-1-7
9653,650.9,1-1-2270008000007,1-1-7
...,...,...,...
397600,409,9-3-0000000994002,9-3-5222
397601,509,9-3-0000000995696,9-3-138
397602,114.99,9-3-0000000997096,9-3-5222
397603,219,9-3-0000000997126,9-3-5222


In [34]:
# le agregamos el idSemana
filename = 'precios_semana_20200503.json'
# Lo obtenemos del nombre del archivo
filename_split = filename.split('_')[2].split('.')
semana = filename_split[0]
tipo = filename_split[1]

In [35]:
# agregamos semanaId
precios_0503['semanaId'] = semana
precios_0503

,precio,producto_id,sucursal_id,semanaId
0,29.9,0000000002288,2-1-187,20200503
1,39.9,0000000002288,2-3-247,20200503
2,499.99,0000000205870,9-1-685,20200503
3,539.99,0000000205870,9-2-22,20200503
4,519.99,0000000205870,9-2-59,20200503
...,...,...,...,...
397729,139.99,9569753142128,25-1-1,20200503
397730,34.99,9795403001143,25-1-1,20200503
397731,312.5,9990385651922,5-1-5,20200503
397732,312.5,9990385651939,5-1-5,20200503


### precios_semana_20200518.txt

In [36]:
# no hay nulos en esta
precios_0518 = pd.read_csv('../datasets/precios_semana_20200518.txt', sep='|')
precios_0518.head()

,precio,producto_id,sucursal_id
0,29.90,0000000002288,2-1-009
1,32.90,0000000002288,2-1-037
2,36.90,0000000002288,2-1-090
3,39.90,0000000002288,2-3-247
4,499.99,0000000205870,9-1-430


In [37]:
precios_0518.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 415293 entries, 0 to 415292
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   precio       413333 non-null  float64
 1   producto_id  415287 non-null  object 
 2   sucursal_id  415287 non-null  object 
dtypes: float64(1), object(2)
memory usage: 9.5+ MB


In [38]:
# elimino filas con nulos en sucursal y producto
index = precios_0518.loc[(precios_0518.producto_id.isnull()) & (precios_0518.sucursal_id.isnull())].index
precios_0518.drop(index, axis='index', inplace=True)

# reviso que no hay nulos
precios_0518.loc[(precios_0518.producto_id.isnull()) & (precios_0518.sucursal_id.isnull())]

,precio,producto_id,sucursal_id


In [39]:
precios_0518.loc[precios_0518.precio.isnull()]

,precio,producto_id,sucursal_id
8656,NaN,0643131504438,65-1-328
8982,NaN,0655257738062,65-1-328
8983,NaN,0655257738246,65-1-328
18050,NaN,3014260014445,65-1-328
18082,NaN,3014260019723,65-1-328
...,...,...,...
409962,NaN,8004200128702,65-1-328
410003,NaN,8004200128801,65-1-328
410043,NaN,8004200128900,65-1-328
410071,NaN,8004200129006,65-1-328


In [40]:
# le agregamos el idSemana
filename = 'precios_semana_20200518.txt'
# Lo obtenemos del nombre del archivo
filename_split = filename.split('_')[2].split('.')
semana = filename_split[0]
tipo = filename_split[1]

In [41]:
# agregamos semanaId
precios_0518['semanaId'] = semana
precios_0518

,precio,producto_id,sucursal_id,semanaId
0,29.90,0000000002288,2-1-009,20200518
1,32.90,0000000002288,2-1-037,20200518
2,36.90,0000000002288,2-1-090,20200518
3,39.90,0000000002288,2-3-247,20200518
4,499.99,0000000205870,9-1-430,20200518
...,...,...,...,...
415288,149.99,9569753142128,25-1-1,20200518
415289,34.99,9795403001143,25-1-1,20200518
415290,312.50,9990385651922,5-1-4,20200518
415291,312.50,9990385651939,5-1-4,20200518


### Juntamos precios

### Tratamiento de nulos
Las tablas con nulos son 0419, 0426 y 0518

In [42]:
# pero el porcentaje de nulos es muy bajo
round((precios_0419.loc[precios_0419.precio.isnull()].shape[0]/precios_0419.shape[0])*100,2)

0.39

In [43]:
# Tabla auxiliar con precios nulos
precios_aux = precios_0419.loc[precios_0419.precio.isnull()].copy()
precios_aux['tipoError'] = 0
precios_aux.head()

,precio,sucursal_id,producto_id,semanaId,tipoError
10045,NaN,65-1-341,0643131504438,20200419,0
10426,NaN,65-1-341,0655257738062,20200419,0
10427,NaN,65-1-341,0655257738246,20200419,0
20608,NaN,65-1-341,3014260014445,20200419,0
20640,NaN,65-1-341,3014260019723,20200419,0


In [44]:
# los relleno con la moda
precios_0419.loc[precios_0419.precio.isnull(),'precio'] = precios_0419.precio.mode()[0]
precios_0419.loc[precios_0419.precio.isnull()]

,precio,sucursal_id,producto_id,semanaId


In [45]:
# agregamos los que tienen precios faltantes al aux
df = precios_0426.loc[precios_0426.precio.isnull()].copy()
df['tipoError'] = 0
precios_aux = pd.concat([precios_aux, df])
del df
precios_aux.head()

,precio,sucursal_id,producto_id,semanaId,tipoError
10045,NaN,65-1-341,0643131504438,20200419,0
10426,NaN,65-1-341,0655257738062,20200419,0
10427,NaN,65-1-341,0655257738246,20200419,0
20608,NaN,65-1-341,3014260014445,20200419,0
20640,NaN,65-1-341,3014260019723,20200419,0


In [46]:
# pero el porcentaje de nulos es muy bajo
round((precios_0426.loc[precios_0426.precio.isnull()].shape[0]/precios_0426.shape[0])*100,2)

0.36

In [47]:
# los relleno con la moda
precios_0426.loc[precios_0426.precio.isnull(),'precio'] = precios_0426.precio.mode()[0]
precios_0426.loc[precios_0426.precio.isnull()]

,precio,sucursal_id,producto_id


In [48]:
(precios_0518.loc[precios_0518.precio.isnull()].shape[0]/precios_0518.shape[0])*100

0.47099957378872925